In [433]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

dataset = pd.read_csv('/home/pedro/Documentos/Kaggle_DSs/full_data.csv')

y = dataset.iloc[:,-1]
X = dataset.iloc[:,:-1] 
from imblearn import under_sampling, over_sampling

X = pd.concat([X[['age','hypertension','heart_disease','avg_glucose_level','bmi']],pd.get_dummies(data=X[['gender','work_type','ever_married','smoking_status','Residence_type']])],1)
X[['age','avg_glucose_level','bmi']] = RobustScaler().fit_transform(X[['age','avg_glucose_level','bmi']])
X, y = over_sampling.SMOTE(0.5).fit_resample(X=X,y=y)

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

/tmp/ipykernel_5444/2393603853.py:17: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X = pd.concat([X[['age','hypertension','heart_disease','avg_glucose_level','bmi']],pd.get_dummies(data=X[['gender','work_type','ever_married','smoking_status','Residence_type']])],1)
/home/pedro/anaconda3/envs/dc_claas/lib/python3.9/site-packages/imblearn/utils/_validation.py:586: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [435]:
X_train = torch.from_numpy(X_train.values.astype(np.float32))
X_test = torch.from_numpy(X_test.values.astype(np.float32))
y_train = torch.from_numpy(np.array(y_train))
y_test = torch.from_numpy(np.array(y_test))

In [436]:
class tensor_ds(object):
    def __init__(self,tensor:torch.Tensor):
        self.X = tensor[:,:-1]
        self.y = tensor[:,-1]
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self,values):
        return self.X[values], self.y[values]

tensor_train = torch.concat((X_train, y_train.unsqueeze(1)), dim=1)

train_loader = DataLoader(dataset=tensor_ds(tensor_train), batch_size=math.floor(X_train.shape[0]/100), shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self,input_size,output_size):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, math.floor(input_size/5)),
            nn.ReLU(),
            nn.Linear(math.floor(input_size/5), math.floor(input_size/10)),
            nn.ReLU(),
            nn.Linear(math.floor(input_size/10), output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        logits = nn.Sigmoid()(logits)
        return logits

model = NeuralNetwork(X_train.shape[1],2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [437]:
import time
for i in range(20):
    for index, (X,y) in enumerate(train_loader):
        
        prediction = model(X)

        loss = criterion(prediction,y.type(torch.long))
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        print(f'this is the iteration number {index} - loss = {loss}')
            

this is the iteration number 0 - loss = 0.6602738499641418
this is the iteration number 1 - loss = 0.675442636013031
this is the iteration number 2 - loss = 0.6784647703170776
this is the iteration number 3 - loss = 0.6661836504936218
this is the iteration number 4 - loss = 0.6570139527320862
this is the iteration number 5 - loss = 0.662989616394043
this is the iteration number 6 - loss = 0.6629227995872498
this is the iteration number 7 - loss = 0.6876126527786255
this is the iteration number 8 - loss = 0.6596928834915161
this is the iteration number 9 - loss = 0.7031499147415161
this is the iteration number 10 - loss = 0.6564534902572632
this is the iteration number 11 - loss = 0.6719925999641418
this is the iteration number 12 - loss = 0.6469303965568542
this is the iteration number 13 - loss = 0.6625171899795532
this is the iteration number 14 - loss = 0.6561751961708069
this is the iteration number 15 - loss = 0.6686921119689941
this is the iteration number 16 - loss = 0.693900704

In [438]:
_, indices = torch.max(prediction,1)

In [439]:
indices, y

(tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0]),
 tensor([0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
         0., 0., 0., 0., 0.]))

In [440]:
_, indices = torch.max(model(X_test),1)

In [441]:
indices

tensor([1, 1, 0,  ..., 1, 0, 0])

In [426]:
np.sum([(v == y_test[i]).numpy() for i,v  in enumerate(indices)])/len(y_test)

0.8098591549295775

In [442]:
tensor_test = torch.concat((X_test, y_test.unsqueeze(1)), dim=1)

testloader = DataLoader(dataset=tensor_ds(tensor_train), batch_size=math.floor(X_train.shape[0]/100), shuffle=True)

In [ ]:
correct_pred = {classname: 0 for classname in ['no_avc','avc']}
total_pred = {classname: 0 for classname in ['no_avc','avc']}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')